In [58]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [59]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN'], axis=1)

In [60]:
# Determine the number of unique values in each column.
print(application_df.nunique())

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [61]:
# Instead of APPLICATION_TYPE for binning, use NAME
name_counts = application_df['NAME'].value_counts()
name_counts

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64

In [62]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff = 5
# use the variable name `application_types_to_replace`
names_to_replace = name_counts[name_counts < cutoff].index.tolist()

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

NAME
Other                                                                19803
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
                                                                     ...  
GUILDS OF THE SANTA FE OPERA INC                                         5
FAMILY MOTOR COACH ASSOCIATION                                           5
JAMESTOWN SOCIETY                                                        5
MOST WORSHIPFUL PRINCE HALL GRAND LODGE OF FREE & ACCEPTED MASONS        5
PTA HAWAII CONGRESS                                                      5
Name: count, Length: 403, dtype: int64

In [63]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [64]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff = 1000
# use the variable name `application_types_to_replace`
application_types_to_replace = app_type_counts[app_type_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [65]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [66]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_over1 = classification_counts[classification_counts > 1]
classification_counts_over1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [67]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_cutoff = 250
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts < classifications_cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [68]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [69]:
print(application_df.columns.tolist())

['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'NAME_AACE INTERNATIONAL', 'NAME_ACE MENTOR PROGRAM OF AMERICA INC', 'NAME_ACTS MINISTRY', 'NAME_ACTS MISSIONS', 'NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS', 'NAME_AIR FORCE ASSOCIATION', 'NAME_ALABAMA FEDERATION OF WOMENS CLUBS', 'NAME_ALABAMA TREASURE FOREST ASSOCIATION', 'NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION', 'NAME_ALPHA PHI OMEGA', 'NAME_ALPHA PHI SIGMA', 'NAME_ALPHA PHI SIGMA INC', 'NAME_ALPHA SIGMA PHI FRATERNITY INC', 'NAME_ALTRUSA INTERNATIONAL FOUNDATION INC', 'NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES', 'NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC', 'NAME_AMERICAN ART THERAPY ASSOCIATION INC', 'NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN', 'NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN INC', 'NAME_AMERICAN CHEMICAL SOCIETY', 'NAME_AMERICAN CHRISTIAN FICTION WRITERS INC', 'NAME_AMERICAN COLLEGE OF HEALTHCARE EXECUTIVES', 'NAME_AMERICAN COLLEGE OF NURSE-MIDWIVES', 'NAME_AMERICAN FE

In [70]:
# Split our preprocessed data into our features and target arrays
target = application_df['IS_SUCCESSFUL']
features = application_df.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42, test_size=0.5)

In [71]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [72]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu6', input_dim=len(X_train.columns)))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation='relu6'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                13380     
                                                                 
 dense_4 (Dense)             (None, 20)                620       
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 14021 (54.77 KB)
Trainable params: 14021 (54.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
#create callback every 5 epochs
checkpoint_path = "model2_checkpoint/checkpoint_{epoch:02d}"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5
)

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [74]:
# Train the model
model = nn.fit(X_train_scaled, y_train, epochs=10, callbacks=[checkpoint_callback], verbose=1)

Epoch 1/10
536/536 [==============================] - 1s 2ms/step - loss: 0.5016 - accuracy: 0.7609
Epoch 2/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4297 - accuracy: 0.7948
Epoch 3/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4218 - accuracy: 0.7964
Epoch 4/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4188 - accuracy: 0.8001
Epoch 5/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4170 - accuracy: 0.8008
Epoch 6/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4153 - accuracy: 0.8002
Epoch 7/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4142 - accuracy: 0.8023
Epoch 8/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4133 - accuracy: 0.8039
Epoch 9/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4132 - accuracy: 0.8038
Epoch 10/10
536/536 [==============================] - 1s 2ms/step - loss: 0.4122 - accuracy: 0.8037

In [78]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

536/536 - 0s - loss: 0.4444 - accuracy: 0.7892 - 183ms/epoch - 342us/step
Loss: 0.44435885548591614, Accuracy: 0.7892128229141235


In [76]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Improved_Model.h5")

In [77]:
# previous model results: loss: 0.5423 - accuracy: 0.7340
# current model results: loss: 0.4444 - accuracy: 0.7892
# ~5% improvement